In [1]:
datapath = '/Users/apple/Desktop/big_data/project/data/'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import pyspark
#from pyspark.mllib.stat import Statistics
%matplotlib inline

In [4]:
# Crime Count by ZIPCODE
restaurant = pd.read_csv(datapath + 'crime_with_zipcode.csv')

CParserError: Error tokenizing data. C error: Expected 28 fields in line 32, saw 29


In [3]:
restaurant = pd.read_csv(datapath+'DOHMH_New_York_City_Restaurant_Inspection_Results.csv')
other_business = pd.read_csv(datapath+'Legally_Operating_Businesses.csv')

/Users/apple/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
restaurant.columns

Index(['CAMIS', 'DBA', 'BORO', 'BUILDING', 'STREET', 'ZIPCODE', 'PHONE',
       'CUISINE DESCRIPTION', 'INSPECTION DATE', 'ACTION', 'VIOLATION CODE',
       'VIOLATION DESCRIPTION', 'CRITICAL FLAG', 'SCORE', 'GRADE',
       'GRADE DATE', 'RECORD DATE', 'INSPECTION TYPE'],
      dtype='object')

In [5]:
other_business = other_business[['License Category', 'Business Name','Address ZIP','Address Borough','Detail','Longitude','Latitude']]

In [6]:
other_business.columns

Index(['License Category', 'Business Name', 'Address ZIP', 'Address Borough',
       'Detail', 'Longitude', 'Latitude'],
      dtype='object')

# Restaurant
### Reminder:
* Time period 
  
  Our crime data covered all records from 2006-2016. However this dataset contains every sustained or not yet adjudicated violation citation from every full or special program inspection conducted up to three years prior to the most recent inspection for restaurants and college cafeterias in an active status on the RECORD DATE(2012 -2017).
  
  Keep in mind that restaurants go in and out of business; only restaurants in an active status are included in the dataset.
  
### Possible relationships
1. CUISINE DESCRIPTION with Crime 
2. SCORE mean score Inspection

In [7]:
# Time period
restaurant_data = restaurant[['INSPECTION DATE','GRADE DATE', 'RECORD DATE']]
INSPECTION = restaurant_data['INSPECTION DATE'].copy()
year = list(map(lambda x: x[-4:],INSPECTION.unique()))
#Establishments with inspection date of 1/1/1900 are new establishments that have not yet received a full inspection.


In [8]:
restaurant = restaurant[restaurant['ZIPCODE'].notnull()]

In [9]:
restaurant = restaurant[['DBA', 'BORO','ZIPCODE','CUISINE DESCRIPTION','VIOLATION CODE','VIOLATION DESCRIPTION','SCORE']]
restaurant['ZIPCODE'] = restaurant['ZIPCODE'].astype(int)

In [10]:
restaurant_amount = restaurant.groupby(['ZIPCODE'])['DBA'].count()
violation_score = restaurant.groupby(['ZIPCODE'])['SCORE'].mean()

In [11]:
restaurant_by_zipcode = pd.DataFrame([restaurant_amount,violation_score]).T
restaurant_by_zipcode = restaurant_by_zipcode.rename(columns = {'DBA':'restaurant_amount','SCORE':'violation_score'})

In [12]:
# description type
CUISINE = pd.DataFrame(restaurant.groupby(['ZIPCODE','CUISINE DESCRIPTION'])['DBA'].count())

In [14]:
all_cuisine = restaurant['CUISINE DESCRIPTION'].unique()
for type_re in all_cuisine:
    restaurant[type_re] = restaurant['CUISINE DESCRIPTION'] == type_re
    restaurant_by_zipcode[type_re] = restaurant.groupby(['ZIPCODE'])[type_re].sum()

In [15]:
restaurant_by_zipcode.to_csv(datapath+'restaurant_summary.csv')

#  other_business

In [16]:
other_business['License Category'].unique()

array(['Home Improvement Contractor', 'Stoop Line Stand', 'Sidewalk Cafe',
       'Electronics Store', 'Home Improvement Salesperson',
       'Tow Truck Driver', 'Pawnbroker', 'Electronic & Appliance Service',
       'Locksmith', 'Cigarette Retail Dealer',
       'Secondhand Dealer - General', 'Laundry',
       'Process Server Individual', 'Newsstand',
       'Motion Picture Projectionist', 'Debt Collection Agency', 'Garage',
       'Amusement Device Portable', 'Laundry Jobber', 'Cabaret',
       'Sightseeing Bus', 'Tow Truck Company', 'General Vendor',
       'Pedicab Driver', 'Ticket Seller', 'Horse Drawn Driver',
       'Amusement Arcade', 'Amusement Device Temporary',
       'Pedicab Business', 'Secondhand Dealer - Auto',
       'Pool or Billiard Room', 'Process Serving Agency',
       'Storage Warehouse', 'Scrap Metal Processor', 'Parking Lot',
       'Dealer In Products', 'Amusement Device Permanent', 'Laundries',
       'Special Sale', 'Bingo Game Operator', 'Games of Chance',
 

In [17]:
parking = other_business[list(map(lambda x: x in ['Garage and Parking Lot','Parking Lot'],other_business['License Category']))]

In [103]:
parking['Address Borough'].unique()

array(['Brooklyn', 'Manhattan', 'Queens', 'Bronx', nan, 'Staten Island'], dtype=object)

In [121]:
parking_boro = parking[parking['Address Borough'] == 'Manhattan'].dropna()
other_boro = other_business[list(map(lambda x: x not in ['Garage and Parking Lot','Parking Lot'],other_business['License Category']))]\
                             [other_business['Address Borough'] == 'Manhattan'].dropna()

/Users/apple/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  from ipykernel import kernelapp as app


# crime

In [30]:
data = pd.read_csv('/Users/apple/Desktop/big_data/project/data/NYPD_Complaint_Data_Historic.csv')

/Users/apple/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [56]:
data_location = data[['BORO_NM','OFNS_DESC','LAW_CAT_CD','Latitude','Longitude']]

In [57]:
data_location = data_location.dropna()

In [152]:
data_location['BORO_NM'].unique()

array(['BRONX', 'QUEENS', 'MANHATTAN', 'BROOKLYN', 'STATEN ISLAND'], dtype=object)

In [63]:
FELONY = data_location[data_location['LAW_CAT_CD'] == 'FELONY']

In [120]:
FELONY_borough = data_location[data_location['BORO_NM']=='MANHATTAN'][data_location['LAW_CAT_CD'] == 'FELONY']

/Users/apple/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


# MAKE MAP
http://sensitivecities.com/so-youd-like-to-make-a-map-using-python-EN.html#.WQ0Wz2NflsI

https://github.com/pbugnion/gmaps

http://qingkaikong.blogspot.com/2016/02/plot-earthquake-heatmap-on-basemap-and.html

In [123]:
def map_business_crime(crime,business,area,crime_type,business_main_flag = False, business_main=None):
    gmap = gmplot.GoogleMapPlotter(40.734714, -73.943641, 14)
    gmap.heatmap(crime.Latitude, crime.Longitude,threshold=100,radius=20)
    gmap.scatter(business.Latitude, business.Longitude,'k',marker=False, size=10)
    if business_main_flag:
        gmap.scatter(business_main.Latitude, business_main.Longitude, 'b',size=20, marker=False)
    gmap.draw('/Users/apple/Desktop/big_data/project/figure/maps/map_'+crime_type+'_'+area+'.html')
import gmplot

map_business_crime(FELONY_borough,other_boro,'Manhattan','Felony',True,parking_boro)

In [113]:
gmap.heatmap(FELONY.Latitude, FELONY.Longitude,threshold=1000,radius=10)
gmap.draw('/Users/apple/Desktop/big_data/project/figure/maps/heat_map_crime_all.html')

In [158]:
len(FELONY)
#len(MISDEMEANOR)
#len(VIOLATION)
FELONY = data_location[data_location['LAW_CAT_CD'] == 'FELONY'][data_location['BORO_NM']=='BROOKLYN']
MISDEMEANOR = data_location[data_location['LAW_CAT_CD'] == 'MISDEMEANOR'][data_location['BORO_NM']=='BROOKLYN']
VIOLATION = data_location[data_location['LAW_CAT_CD'] == 'VIOLATION'][data_location['BORO_NM']=='BROOKLYN']


/Users/apple/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/apple/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/apple/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [159]:
# map of three type of crime

gmap = gmplot.GoogleMapPlotter(40.753918, -73.983951, 12)

gmap.heatmap(FELONY.Latitude, FELONY.Longitude,threshold=400,radius=10)
#gmap.scatter(MISDEMEANOR.Latitude, MISDEMEANOR.Longitude,'b',marker=False, size=10)
#gmap.scatter(VIOLATION.Latitude, VIOLATION.Longitude,'m',marker=False, size=10)

gmap.draw('/Users/apple/Desktop/big_data/project/figure/maps/LAW_CAT_CD_compare_BROOKLYN_FELONY.html')


In [160]:
gmap = gmplot.GoogleMapPlotter(40.753918, -73.983951, 12)

gmap.heatmap(MISDEMEANOR.Latitude, MISDEMEANOR.Longitude,threshold=400,radius=10)
#gmap.scatter(MISDEMEANOR.Latitude, MISDEMEANOR.Longitude,'b',marker=False, size=10)
#gmap.scatter(VIOLATION.Latitude, VIOLATION.Longitude,'m',marker=False, size=10)

gmap.draw('/Users/apple/Desktop/big_data/project/figure/maps/LAW_CAT_CD_compare_BROOKLYN_MISDEMEANOR.html')

gmap = gmplot.GoogleMapPlotter(40.753918, -73.983951, 12)

gmap.heatmap(VIOLATION.Latitude, VIOLATION.Longitude,threshold=400,radius=10)
#gmap.scatter(MISDEMEANOR.Latitude, MISDEMEANOR.Longitude,'b',marker=False, size=10)
#gmap.scatter(VIOLATION.Latitude, VIOLATION.Longitude,'m',marker=False, size=10)

gmap.draw('/Users/apple/Desktop/big_data/project/figure/maps/LAW_CAT_CD_compare_BROOKLYN_VIOLATION.html')



# transform to zipcode

In [162]:
data_location[:10]

,BORO_NM,OFNS_DESC,LAW_CAT_CD,Latitude,Longitude
0,BRONX,FORGERY,FELONY,40.828848,-73.916661
1,QUEENS,MURDER & NON-NEGL. MANSLAUGHTER,FELONY,40.697338,-73.784557
2,MANHATTAN,DANGEROUS DRUGS,FELONY,40.802607,-73.945052
3,QUEENS,ASSAULT 3 & RELATED OFFENSES,MISDEMEANOR,40.654549,-73.726339
4,MANHATTAN,ASSAULT 3 & RELATED OFFENSES,MISDEMEANOR,40.738002,-73.987891
5,BROOKLYN,FELONY ASSAULT,FELONY,40.665023,-73.957111
6,MANHATTAN,DANGEROUS DRUGS,MISDEMEANOR,40.720200,-73.988735
7,BRONX,DANGEROUS WEAPONS,FELONY,40.845707,-73.910398
8,BRONX,ASSAULT 3 & RELATED OFFENSES,MISDEMEANOR,40.856711,-73.891900
9,MANHATTAN,PETIT LARCENY,MISDEMEANOR,40.765618,-73.963623


In [207]:
from geopy.geocoders import Nominatim
def transform_to_zipcode(string):
    geolocator = Nominatim()
    location = geolocator.reverse(string)
    return location.raw['address']['postcode']

In [188]:
transform_to_zipcode('40.828848, -73.916661')

10456

In [189]:
zipcode = dict.fromkeys(list(data_location.index))
for x in zipcode.keys():
    zipcode[x] = 0

4894911

In [212]:
0 in zipcode.values()

True

In [240]:
p=pd.Series(zipcode)

In [241]:
len(p)- sum(p==0)


2864

In [243]:
p.to_csv(datapath+'crime_zipcode.csv')

In [235]:
from geopy.geocoders.base import GeocoderServiceError
from urllib.request import URLError

In [239]:
import time

while 0 in zipcode.values():
    try:
        for i in data_location.index:
            if zipcode[i]==0:
                zipcode[i]=transform_to_zipcode(str(data_location.ix[i]['Latitude'])+', '+str(data_location.ix[i]['Longitude']))
    except (URLError,OSError,GeocoderServiceError ):
        time.sleep(10)
        

KeyboardInterrupt: 

In [181]:
data_location['zipcode'] = list(map(lambda x,y: transform_to_zipcode(str(x)+', '+str(y)), 
                                    data_location.Latitude,data_location.Longitude))

GeocoderServiceError: [Errno 65] No route to host

In [183]:
data_location_t = data_location.dropna()